# Défi IA 2023 : Prédiction du log_price avec Random Forest

## Packages utiles

In [95]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from math import sqrt, log
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import GridSearchCV

## Prediction du log_price avec Random Forest

In [96]:
data_train=pd.read_csv('./Datatrain/data_train_final.csv')
data_train=data_train.drop(['Unnamed: 0'],axis=1)

In [97]:
data_train

,index,hotel_id,price,stock,city,date,language,mobile,avatar_id,index_request,group,brand,parking,pool,children_policy
0,0,639,149,110,valletta,35,greek,1,155054,1.0,Yin Yang,Royal Lotus,0,1,0
1,1,376,159,115,valletta,35,greek,1,155054,1.0,Accar Hotels,Marcure,0,0,0
2,2,985,68,70,valletta,35,greek,1,155054,1.0,Independant,Independant,1,0,0
3,3,377,127,84,valletta,35,greek,1,155054,1.0,Boss Western,J.Halliday Inn,1,0,0
4,4,200,130,80,valletta,35,greek,1,155054,1.0,Boss Western,J.Halliday Inn,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351748,291402,620,170,12,vilnius,15,swedish,1,146515,0.0,Chillton Worldwide,Quadrupletree,0,1,0
351749,291403,605,182,15,vilnius,15,swedish,1,146515,0.0,Yin Yang,Royal Lotus,1,0,0
351750,291404,516,264,23,vilnius,15,swedish,1,146515,0.0,Morriott International,Corlton,1,0,2
351751,291405,490,144,13,vilnius,15,swedish,1,146515,0.0,Yin Yang,Ardisson,1,0,0


In [98]:
train_sorted_bis=pd.read_csv('./Datatrain/train_sorted_bis.csv')
train_sorted_bis=train_sorted_bis.drop(['Unnamed: 0'],axis=1)

In [99]:
train_sorted_bis

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy,is_test,p
0,163.303989,37,160.713324,0,493,82,181.882663,156.543407,0,0,0,0,0.01
1,178.845041,6,161.386134,0,225,4,146.789097,153.026801,1,0,0,0,0.01
2,163.303989,37,161.784635,1,256,37,174.788016,127.821561,1,0,0,0,0.01
3,154.264554,37,159.239684,0,512,119,154.236450,180.153032,1,1,0,0,0.01
4,153.781035,6,154.085495,1,854,6,174.788016,225.626198,0,1,0,0,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57334,157.686331,1,161.784635,1,428,0,146.789097,137.730454,1,0,0,0,0.57
57335,163.303989,1,156.021126,0,441,1,154.236450,180.153032,0,0,0,0,0.58
57336,163.303989,1,158.271256,1,917,1,187.146369,216.925345,1,1,0,0,0.60
57337,154.264554,1,162.909176,0,504,1,187.146369,174.426454,0,0,0,0,0.62


In [100]:
data_train["log_price"]=data_train["price"].map(lambda x : log(x))
data_train.head()

,index,hotel_id,price,stock,city,date,language,mobile,avatar_id,index_request,group,brand,parking,pool,children_policy,log_price
0,0,639,149,110,valletta,35,greek,1,155054,1.0,Yin Yang,Royal Lotus,0,1,0,5.003946
1,1,376,159,115,valletta,35,greek,1,155054,1.0,Accar Hotels,Marcure,0,0,0,5.068904
2,2,985,68,70,valletta,35,greek,1,155054,1.0,Independant,Independant,1,0,0,4.219508
3,3,377,127,84,valletta,35,greek,1,155054,1.0,Boss Western,J.Halliday Inn,1,0,0,4.844187
4,4,200,130,80,valletta,35,greek,1,155054,1.0,Boss Western,J.Halliday Inn,0,0,0,4.867534


In [101]:
list_avatar_train=data_train['avatar_id'].unique()

In [102]:
data_train['index_request']=0

In [103]:
for avatar in list_avatar_train : 
    test1=data_train[data_train['avatar_id'] ==avatar].copy()
    list_date=test1['date'].unique()
    
    if len(list_date)==1 : 
        ech=test1[test1['date'] ==list_date[0]].copy()
        ech['index_request']=1
        data_train[data_train['avatar_id']==avatar]=ech
        
    else : 
        for i in range(len(list_date)) : 
            ech=test1[test1['date'] ==list_date[i]].copy()
            ech['index_request']=i+1
            test1[test1['date'] ==list_date[i]]=ech
        data_train[data_train['avatar_id']==avatar]=test1

In [104]:
data_train

,index,hotel_id,price,stock,city,date,language,mobile,avatar_id,index_request,group,brand,parking,pool,children_policy,log_price
0,0,639,149,110,valletta,35,greek,1,155054,1,Yin Yang,Royal Lotus,0,1,0,5.003946
1,1,376,159,115,valletta,35,greek,1,155054,1,Accar Hotels,Marcure,0,0,0,5.068904
2,2,985,68,70,valletta,35,greek,1,155054,1,Independant,Independant,1,0,0,4.219508
3,3,377,127,84,valletta,35,greek,1,155054,1,Boss Western,J.Halliday Inn,1,0,0,4.844187
4,4,200,130,80,valletta,35,greek,1,155054,1,Boss Western,J.Halliday Inn,0,0,0,4.867534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351748,291402,620,170,12,vilnius,15,swedish,1,146515,3,Chillton Worldwide,Quadrupletree,0,1,0,5.135798
351749,291403,605,182,15,vilnius,15,swedish,1,146515,3,Yin Yang,Royal Lotus,1,0,0,5.204007
351750,291404,516,264,23,vilnius,15,swedish,1,146515,3,Morriott International,Corlton,1,0,2,5.575949
351751,291405,490,144,13,vilnius,15,swedish,1,146515,3,Yin Yang,Ardisson,1,0,0,4.969813


In [105]:
data_test= pd.read_csv('test_set.csv')
data_test.head()

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock
0,0,1,vilnius,21,romanian,0,1,161,46
1,1,1,vilnius,21,romanian,0,1,187,32
2,2,1,vilnius,21,romanian,0,1,279,12
3,3,1,vilnius,21,romanian,0,1,395,10
4,4,1,vilnius,21,romanian,0,1,488,42


In [106]:
list_avatar_test=data_test['avatar_id'].unique()

In [107]:
data_test['index_request']=0

In [108]:
for avatar in list_avatar_test : 
    test1=data_test[data_test['avatar_id'] ==avatar].copy()
    list_date=test1['date'].unique()
    
    if len(list_date)==1 : 
        ech=test1[test1['date'] == list_date[0]].copy()
        ech['index_request']=1
        data_test[data_test['avatar_id']==avatar]=ech
        
    else : 
        for i in range(len(list_date)) : 
            ech=test1[test1['date'] ==list_date[i]].copy()
            ech['index_request']=i+1
            test1[test1['date'] ==list_date[i]]=ech
        data_test[data_test['avatar_id']==avatar]=test1

In [109]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
data_test = data_test.join(hotels, on=['hotel_id', 'city'])
data_test.head()

,index,order_requests,city,date,language,mobile,avatar_id,hotel_id,stock,index_request,group,brand,parking,pool,children_policy
0,0,1,vilnius,21,romanian,0,1,161,46,1,Boss Western,J.Halliday Inn,1,0,0
1,1,1,vilnius,21,romanian,0,1,187,32,1,Accar Hotels,Marcure,1,1,0
2,2,1,vilnius,21,romanian,0,1,279,12,1,Independant,Independant,1,0,0
3,3,1,vilnius,21,romanian,0,1,395,10,1,Accar Hotels,Ibas,0,0,0
4,4,1,vilnius,21,romanian,0,1,488,42,1,Accar Hotels,Safitel,1,1,1


In [110]:
# this function is by Max Halford at the address noted above
def calc_smooth_mean(df, by, on, m, target_df):
    '''Function returns a weighted mean value for the each member of a column.
    Arguments:
    df: The df being used to calculate the means
    by: the column being target encoded
    on: the thing to be encoded; almost always price in this circumstance
    m: weight before moving toward global mean; usually a min # samples
    target_df: the target df for the mean encoding. Could be same as df or different.'''
    # Compute the global mean
    mean = df[on].mean() 

    # Compute the number of values and the mean of each group
    agg = df.groupby(by)[on].agg(['count', 'mean'])  
    counts = agg['count']
    means = agg['mean']

    # Compute the "smoothed" means
    smooth = (counts * means + m * mean) / (counts + m)

    # Replace each value by the according smoothed mean
    return target_df[by].map(smooth)
    #return round(target_df[by].map(smooth), 0) 

In [111]:
# get size of training data
num_of_samples = data_train.shape[0]

# determining minimum number of samples for zip and month to use their
# own mean rather than expanding into the full data set mean 
city_samples = num_of_samples/data_train['city'].unique().shape[0]
group_samples = num_of_samples/data_train['group'].unique().shape[0]
brand_samples = num_of_samples/data_train['brand'].unique().shape[0]
language_samples = num_of_samples/data_train['language'].unique().shape[0]

In [112]:
agg = data_train.groupby('city')['price'].agg(['count', 'mean'])  
counts = agg['count']
means = agg['mean']
mean = data_train['price'].mean()
smooth_city = (counts * means + city_samples * mean) / (counts + city_samples)

In [113]:
agg = data_train.groupby('group')['price'].agg(['count', 'mean'])  
counts = agg['count']
means = agg['mean']
mean = data_train['price'].mean()
smooth_group = (counts * means + group_samples * mean) / (counts + group_samples)


In [114]:
agg = data_train.groupby('brand')['price'].agg(['count', 'mean'])  
counts = agg['count']
means = agg['mean']
mean = data_train['price'].mean()
smooth_brand = (counts * means + brand_samples * mean) / (counts + brand_samples)

In [115]:
agg = data_train.groupby('language')['price'].agg(['count', 'mean'])  
counts = agg['count']
means = agg['mean']
mean = data_train['price'].mean()
smooth_language = (counts * means + language_samples * mean) / (counts + language_samples)

In [116]:
import pickle

with open("dict_city_encoding.pkl", "wb") as tf:
    pickle.dump(smooth_city.to_dict(),tf)
    
with open("dict_group_encoding.pkl", "wb") as tf:
    pickle.dump(smooth_group.to_dict(),tf)
    
with open("dict_brand_encoding.pkl", "wb") as tf:
    pickle.dump(smooth_brand.to_dict(),tf)
    
with open("dict_language_encoding.pkl", "wb") as tf:
    pickle.dump(smooth_language.to_dict(),tf)

In [117]:
data_test['city'] = calc_smooth_mean(data_train, by='city', on='price', m=city_samples,target_df=data_test)
data_test['group'] = calc_smooth_mean(data_train, by='group', on='price', m=group_samples,target_df=data_test)
data_test['brand'] = calc_smooth_mean(data_train, by='brand', on='price', m=brand_samples,target_df=data_test)
data_test['language'] = calc_smooth_mean(data_train, by='language', on='price', m=language_samples,target_df=data_test)

In [118]:
data_train['city'] = calc_smooth_mean(data_train, by='city', on='price', m=city_samples,target_df=data_train)
data_train['group'] = calc_smooth_mean(data_train, by='group', on='price', m=group_samples,target_df=data_train)
data_train['brand'] = calc_smooth_mean(data_train, by='brand', on='price', m=brand_samples,target_df=data_train)
data_train['language'] = calc_smooth_mean(data_train, by='language', on='price', m=language_samples,target_df=data_train)

In [119]:
data_test=data_test.drop(['index'],axis=1)
data_train=data_train.drop(['index'],axis=1)
data_train=data_train.drop(['avatar_id'],axis=1)
data_test=data_test.drop(['avatar_id'],axis=1)
data_train=data_train.drop(['price'],axis=1)
data_test=data_test.drop(['order_requests'],axis=1)
data_train=data_train.drop(['index_request'],axis=1)
data_test=data_test.drop(['index_request'],axis=1)

In [120]:
cols=['city','date','language','mobile','hotel_id','stock','group','brand','parking','pool','children_policy','log_price']

In [121]:
data_train=data_train[cols]

In [122]:
data_train=data_train.iloc[train_sorted_bis.index]

In [123]:
logprice=data_train['log_price']
X_train, X_test, Y_train, Y_test = train_test_split(data_train,logprice,test_size=0.25,random_state=11)
X_train=X_train.drop(['log_price'],axis=1)

In [124]:
X_train

,city,date,language,mobile,hotel_id,stock,group,brand,parking,pool,children_policy
4281,157.686331,7,159.097269,0,920,0,121.370491,109.032349,0,0,0
50769,136.682873,5,161.386134,1,432,5,187.146369,174.426454,0,0,0
3062,163.303989,29,159.440823,0,193,45,121.370491,109.032349,0,0,0
52686,163.303989,7,161.299514,0,743,4,187.146369,158.986533,1,1,0
49017,180.021377,6,156.021126,1,523,4,121.370491,109.032349,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
32081,178.845041,25,161.697874,0,449,50,154.236450,180.153032,1,0,0
7259,180.021377,43,156.006283,0,481,52,121.370491,109.032349,1,0,0
21584,153.781035,19,161.386134,1,452,26,181.882663,175.895907,0,1,0
36543,163.303989,6,156.021126,1,630,0,146.789097,153.026801,0,0,0


In [125]:
param=[{"max_features":list(range(2,10))}]
regrf= GridSearchCV(RandomForestRegressor(),param,cv=5,n_jobs=-1)
regrfOpt=regrf.fit(X_train, Y_train)
# paramètre optimal
print("Meilleur score = %f, Meilleur paramètre = %s" % (regrfOpt.best_score_,regrfOpt.best_params_))



Meilleur score = 0.993427, Meilleur paramètre = {'max_features': 9}


In [126]:
Ypred_regrf = regrfOpt.predict(data_test)

In [127]:
import pickle

with open("regrfOpt_model.pkl", "wb") as f:
    pickle.dump(regrfOpt, f)  
    
with open("regrfOpt_model.pkl", "rb") as f:
    regrfOpt = pickle.load(f)

In [128]:
d_test= pd.read_csv('test_set.csv')
submission=pd.DataFrame()
submission['index']=d_test['index']
submission['price']=np.exp(Ypred_regrf)

In [129]:
submission.head()

,index,price
0,0,110.421807
1,1,142.236081
2,2,57.947548
3,3,58.097850
4,4,194.361708


In [130]:
submission.to_csv('submission.csv', index=False)